In [93]:
#import nessary libraries

from bs4 import BeautifulSoup as bs
import requests       
import smtplib        #sending emails
import csv
import pandas as pd    #making Dataframes


In [94]:
#Sending mail for users if price is reduced 
import os
from email.message import EmailMessage
import ssl
import smtplib
from secret import sender_email,password


def send_mail():

    email_sender = sender_email
    email_pasword = password
    receiver_email = 'tuanminhajseedin@gmail.com'

    subject='The book you want is below $50! Now is your chance to buy!'

    body="""Tuan, This is the moment we have been waiting for, 
    now is your chance to pick this great chance. Don't miss this chance"""


    em=EmailMessage()
    em['From'] = email_sender
    em['To'] = email_receiver
    em['Subject'] = subject
    em.set_content(body)

    context = ssl.create_default_context()

    with smtplib.SMTP_SSL('smtp.gmail.com',465,context=context) as smtp:
        smtp.login(email_sender,email_pasword)
        smtp.sendmail(email_sender,email_receiver,em.as_string())




    

#### Method 01 for load data into csv

In [98]:
#initiate a csv file to load the scraped data
headers=['Title','Star','Price','Availability']
with open('books_Web_Scraper_Dataset.csv','w',newline='',encoding='UTF8') as f:
    writer=csv.writer(f)
    writer.writerow(headers)

In [117]:
#connecting to the website and scrape all the data from the website and take into local
books=[]
def book_details():
    for i in range(1,51):
        url=f'https://books.toscrape.com/catalogue/page-{i}.html'
        # print(url)

        page=requests.get(url)
        soup1=bs(page.content,"html.parser")   #pulling all the html from the website
        
        #find searching under tag name of ol(order_list) and find by article with required class
        order_list=soup1.find('ol')
        articles=order_list.find_all('article',class_='product_pod')

        
        
        #retrieve all the titles inside the articles and taking the all titles in the first page
        for article in articles:        #looping through each articles
            image=article.find('img')       #find all img tags
            title=image.attrs['alt']        #filter out the title with it attributes
            star=article.find('p')          #star ratings are under p tags we can straight scrape it
            star=star['class'][1]           #to retrieve only rating value we need to get 1 index values
            price=article.find('p',class_='price_color').text     #finding the price values, those are under p tags with class of price_color and filter it's text value
            price=float(price[1:])             #text values are return with strings. we need to change it's type as float to store in csv
            availability=article.find('p',class_='instock availability').text
            availability=availability.strip()    #remove unnessary spaces 

            books.append([title,star,price,availability])
        
            data=[title,star,price,availability]

            with open('books_Web_Scraper_Dataset.csv','a+',newline='',encoding='UTF8') as f:    #appending all the data into the csv file
                writer=csv.writer(f)
                writer.writerow(data)


            if price<50:                      #sending a mail for user if price is reduced
                send_mail()
        
    

#### Method 2 for load data into csv

In [113]:
#making Dataframes
book_details()

df=pd.DataFrame(books,columns=headers)

In [114]:
df.head()

,Title,Star,Price,Availability
0,A Light in the Attic,Three,51.77,In stock
1,Tipping the Velvet,One,53.74,In stock
2,Soumission,One,50.10,In stock
3,Sharp Objects,Four,47.82,In stock
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock


In [115]:
# convert dataframe into csv file format

df.to_csv('books.csv')


In [116]:
#Automate the process in every single day and update the csv file
while True:                           
    book_details()
    time.sleep(3600)

KeyboardInterrupt: 